In [1]:
# Start a simple Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('exercise').getOrCreate()
print('Spark is Connected')

24/01/09 17:59:03 WARN Utils: Your hostname, MaheshPC resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface enp5s0)
24/01/09 17:59:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 17:59:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/09 17:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/09 17:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/01/09 17:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/01/09 17:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/01/09 17:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.

Spark is Connected


In [2]:
# Load walmart dataset
df = spark.read.csv('DatasetExercise/walmart_stock.csv',inferSchema=True,header=True)
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [6]:
# what are the column names
# schema

df.printSchema()
df.columns

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [10]:
# first 5 columns
for i in df.head(5):
    print(i)

Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)


In [11]:
# describe
df.describe().show()

24/01/09 18:04:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [14]:
# to print decimal using format number
from pyspark.sql.functions import format_number
res = df.describe()
res.select(res['summary'],format_number(res['Open'].cast('float'),2).alias('OPEN'),
           format_number(res['Close'].cast('float'),2).alias('Close'),
           format_number(res['High'].cast('float'),2).alias('HIGH'),
           format_number(res['Low'].cast('float'),2).alias('LOW')
         
          
          ).show()

+-------+--------+--------+--------+--------+
|summary|    OPEN|   Close|    HIGH|     LOW|
+-------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|
|   mean|   72.36|   72.39|   72.84|   71.92|
| stddev|    6.77|    6.76|    6.77|    6.74|
|    min|   56.39|   56.42|   57.06|   56.30|
|    max|   90.80|   90.47|   90.97|   89.25|
+-------+--------+--------+--------+--------+



In [17]:
# create new dataframe and make it HV ratio

df2 = df.withColumn("HV RATIO",df['High']/df['Volume'])
df2.select('HV RATIO').show()


+--------------------+
|            HV RATIO|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [18]:
# highest price
df.orderBy(df [ 'High'].desc()) .head (1)[0][0]

datetime.date(2015, 1, 13)

In [19]:
from pyspark.sql.functions import mean
df.select (mean ("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [20]:
"""What percentage of the time was the High greater than 80 dollars ?
(Number of Days High>80)/(Total Days in the dataset)"""

(df.filter (df[ 'High' ]>80) .count()*1.0 / df.count() )* 100

9.141494435612083

In [22]:
# pearson correlation
from pyspark.sql.functions import corr
df.select (corr('High','Volume')) .show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+

